# Experimenting with Custom Tools

This notebook demonstrates how to use the custom tools defined in `src/agents/tools/`:

1. Web Search Tools (`searchweb.py`)
2. Wikipedia Search Tools (`wikisearch.py`)
3. Calculator Tools (`calctools.py`)

Let's explore each tool's functionality.

In [34]:

import sys
from typing import List
from pydantic import BaseModel, Field


# Add src to path for imports
sys.path.append('../src')

# Import the tools
from agents.tools.searchweb import search_web, search_web_with_query, search_web_get_answer, SearchResult, scrape_web_agent
from agents.tools.wikisearch import search_wikipedia, search_wikipedia_with_query
from agents.tools.calctools import calculator

# Define Player model
class Player(BaseModel):
    name: str
    height: str | None = None
    hometown: str | None = None
    position: str | None = None
    velocity: str | None = None
    pg_grade: str | None = None

class Team(BaseModel):
    team_name: str
    players: List[Player]

class RosterURL(BaseModel):
    url: str = Field(description="The official roster URL for the college baseball team")



## 1. Web Search Tools

The web search tools provide several ways to search and scrape web content:

In [4]:
# Example 1: Basic web search with query generation
results = search_web("What is the offical url of the University of Scranton baseball team roster")
print("\nSearch Results:")
for result in results:
    print(f"\nURL: {result.link}")
    print(f"Content: {result.content[:200]}...")

SEARCH QUERY:  University of Scranton baseball team roster site:scranton.edu

Search Results:

URL: https://athletics.scranton.edu/sports/baseball
Content: The Official Athletics Site of The University of Scranton. Main Navigation Menu. Baseball Baseball: Twitter Baseball: Instagram Baseball: Schedule Baseball: Roster Baseball: News Men's Basketball Men'...

URL: https://athletics.scranton.edu/sports/baseball/schedule/2025
Content: The Official Athletics Site of The University of Scranton. Main Navigation Menu. Baseball Baseball: Twitter Baseball: Instagram Baseball: Schedule Baseball: Roster Baseball: News Men's Basketball Men'...

URL: https://athletics.scranton.edu/sports/baseball/roster
Content: The official 2024 Baseball Roster for the University of Scranton 2024 Baseball Roster - University of Scranton Skip To Main Content Pause All Rotators...


In [47]:
college_name = "University of Scranton"

In [52]:
# Example 2: Direct web search with query
results = search_web_with_query(f"What is the offical .edu url of the {college_name} baseball team 2024 or 2025 roster",max_results=5)
print("\nDirect Search Results:")
for result in results:
    print(f"\nURL: {result.link}")
    print(f"Content: {result.content[:200]}...")


Direct Search Results:

URL: https://athletics.scranton.edu/sports/baseball/roster
Content: Fr. L/R Flemington, N.J. Hunterdon Central Freshman L/R Flemington, N.J. Hunterdon Central Sr. L/R East Brunswick, N.J. St. Joseph's Senior L/R East Brunswick, N.J. St. Joseph's Fr. R/R Flemington, N....

URL: https://athletics.scranton.edu/news/2024/12/11/baseball-releases-2025-schedule.aspx
Content: The reigning Landmark Conference Champion University of Scranton baseball team has announced its 2025 schedule. The 2025 slate will feature 40 games, including 12 home games at Volpe Field in Scranton...

URL: https://athletics.scranton.edu/sports/baseball/schedule/2024
Content: The official 2024 Baseball schedule for the University of Scranton...


In [53]:


# Combine results into single context
context = "\n\n".join([f"URL: {r.link}\nContent: {r.content}" for r in results])

# Use scrape_web_agent to process results
roster_info = await scrape_web_agent(
    results[0].link,
    f"Based on the search results, what is the official roster URL for {college_name} baseball team? " 
    "Look for .edu domains and official athletics pages. Provide your confidence level and reasoning.",
    RosterURL
)

print("\nExtracted Roster URL Info:")
print(f"URL: {roster_info.url}")




Extracted Roster URL Info:
URL: https://www.scrantonathletics.com/sports/baseball/roster


In [55]:
from agents.tools.searchweb import scrape_web


doc = scrape_web(roster_info.url)
print(doc.page_content)


ConnectionError: HTTPSConnectionPool(host='www.scrantonathletics.com', port=443): Max retries exceeded with url: /sports/baseball/roster (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1342c5210>: Failed to resolve 'www.scrantonathletics.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [46]:

# Extract roster information using scrape_web_agent
roster = await scrape_web_agent(
    roster_info.url,
    """Extract the full roster information from this webpage. Include:
    - Each player's full name
    - Position
    - Height if available
    - Hometown if available
    Format as a structured team roster.""",
    Team
)

print("\nTeam Roster:")
print(f"\n{roster.team_name}")
print("\nPlayers:")
for player in roster.players:
    print(f"\nName: {player.name}")
    print(f"Position: {player.position}")
    print(f"Height: {player.height}")
    print(f"Hometown: {player.hometown}")




Team Roster:

Arcadia University Knights

Players:

Name: Kevin Wheeler
Position: OF
Height: 6'1"
Hometown: West Chester, PA

Name: Solomon Abdullah
Position: INF
Height: 5'7"
Hometown: Greenville, DE

Name: Griffin Murphy
Position: C
Height: 5'11"
Hometown: Howell, NJ

Name: Jeremy Coccia
Position: P
Height: 5'8"
Hometown: West Chester, PA

Name: Gabe Diaz
Position: INF
Height: 5'11"
Hometown: Fulshear, TX

Name: Noah Stennett
Position: OF
Height: 6'1"
Hometown: West Orange, NJ

Name: Peter Trendafilov
Position: C/IF
Height: 5'9"
Hometown: Andover, NJ

Name: Nick Bon Tempo
Position: C
Height: 5'6"
Hometown: New Milford, CT

Name: Eric Shandler
Position: INF
Height: 5'9"
Hometown: West Chester, PA

Name: Mitchell Bramsway
Position: P
Height: 6'3"
Hometown: Browns Summit, NC

Name: Xavier Sjostedt
Position: P
Height: 5'11"
Hometown: Doylestown, PA

Name: Anthony Rosario
Position: P
Height: 5'11"
Hometown: Long Island City, NY

Name: Andrew Tyszka
Position: P/OF
Height: 6'0"
Hometown: N

## 2. Wikipedia Search Tools

The Wikipedia tools allow searching Wikipedia content:

In [ ]:
# Example 1: Wikipedia search with query generation
wiki_results = search_wikipedia("Tell me about the history of artificial intelligence")
print("\nWikipedia Search Results:")
print(wiki_results['docresults'][0][:500])

In [ ]:
# Example 2: Direct Wikipedia search
wiki_results = search_wikipedia_with_query("Artificial intelligence history")
print("\nDirect Wikipedia Search Results:")
print(wiki_results['docresults'][0][:500])

## 3. Calculator Tools

The calculator tool provides mathematical computation capabilities:

In [ ]:
# Example 1: Basic arithmetic
result = calculator.invoke("2 + 2 * 3")
print(f"2 + 2 * 3 = {result}")

# Example 2: Using mathematical constants
result = calculator.invoke("pi * 2")
print(f"pi * 2 = {result}")

# Example 3: More complex expression
result = calculator.invoke("sin(pi/2) + sqrt(16)")
print(f"sin(pi/2) + sqrt(16) = {result}")